In [17]:
import sys
sys.path.append('/Users/j/Documents/Forme/code/forme-groups')

from src.groups.all import Unit, UnitValue, UnitTypeRef, UnitType, UnitTypePool, Generator, Frozen, freeze

# Create a UnitTypeRef
# ---------------------
unit_type_ref = UnitTypeRef("new_type")
print(unit_type_ref)

# Create a UnitType
# -----------------
unit_type = UnitType(aliases=[unit_type_ref], super_type=UnitTypeRef("string"), prefix="new_type:", suffix=":new_type", separator="")
print(unit_type)

# Create a UnitTypePool
# ---------------------
unit_type_pool = UnitTypePool()
unit_type_pool.add_type(unit_type)
print(unit_type_pool)

# Create a UnitValue
# ------------------
unit_value = UnitValue("haha")
print(unit_value)

# Create a Generator
# ------------------
generator = Generator(unit_type_pool)
print(generator)

# Create a Unit
# -------------
unit = generator.create_unit("new_type", "unit")
print(unit)

# Create a frozen UnitTypeRef
# ---------------------------
print(unit_type_ref.__class__.__name__)
frozen_unit_type_ref = Frozen(unit_type_ref)

print(frozen_unit_type_ref.__dir__())
frozen_unit_type_ref.freeze()
print(frozen_unit_type_ref.__class__.__name__)
print(frozen_unit_type_ref.type_ref)
print(frozen_unit_type_ref.__repr__())

# Create a frozen UnitType
# ------------------------
print(unit_type.__class__.__name__)
frozen_unit_type = Frozen(unit_type)

print(frozen_unit_type.__dir__())
frozen_unit_type.freeze()
print(frozen_unit_type.__class__.__name__)
print(frozen_unit_type)
print(frozen_unit_type.__repr__())

# Create a frozen UnitTypePool
# ----------------------------
print(unit_type_pool.__class__.__name__)
frozen_unit_type_pool = Frozen(unit_type_pool)

print(frozen_unit_type_pool.__dir__())
frozen_unit_type_pool.freeze()
print(frozen_unit_type_pool.__class__.__name__)
print(frozen_unit_type_pool)
print(frozen_unit_type_pool.__repr__())

# Create a frozen UnitValue using the freeze function
# ---------------------------------------------------
unit_value = UnitValue("haha")
print(unit_value.__class__.__name__)
frozen_unit_value = freeze(unit_value)

print(frozen_unit_value.__class__.__name__)
print(frozen_unit_value.value)
print(frozen_unit_value.__dir__())

# Create a frozen object using freeze function
# --------------------------------------------
new_Unit_value = UnitValue("haha_hhahaha")
print(new_Unit_value.__class__.__name__)

frozen_new_Unit_value = freeze(new_Unit_value)
print(frozen_new_Unit_value.__class__.__name__)
print(frozen_new_Unit_value.value)






UnitTypeRef(type_ref='new_type')
UnitType(aliases=[UnitTypeRef(type_ref='new_type')], super_type=UnitTypeRef(type_ref='string'), prefix='new_type:', suffix=':new_type', separator='', function_call=<object object at 0x116199730>)
UnitTypePool(pool={'integer': UnitType(aliases=(UnitTypeRef(type_ref='integer'), UnitTypeRef(type_ref='int')), super_type=UnitTypeRef(type_ref='__RESERVED_INT__'), prefix=None, suffix=None, separator=None, function_call=<class 'int'>), 'float': UnitType(aliases=(UnitTypeRef(type_ref='float'), UnitTypeRef(type_ref='double'), UnitTypeRef(type_ref='FLOAT'), UnitTypeRef(type_ref='DOUBLE')), super_type=UnitTypeRef(type_ref='__RESERVED_FLOAT__'), prefix=None, suffix=None, separator=None, function_call=<class 'float'>), 'string': UnitType(aliases=(UnitTypeRef(type_ref='string'), UnitTypeRef(type_ref='STRING')), super_type=UnitTypeRef(type_ref='__RESERVED_STRING__'), prefix=None, suffix=None, separator=None, function_call=<class 'str'>), 'boolean': UnitType(aliases=(Un

In [51]:
from attrs import define, frozen, field, validate, validators
from typing import Optional, Any, List, Dict, Set

# _system_types = int | float | bool | str | bytes | list | dict | set | type(None)
_system_types = [int, float, bool, str, bytes, list, dict, set, type(None)]
_value_type = str | bytes

@define(slots=True)
class UnitTypeRef:
    type_ref: str


@define(slots=True)
class UnitType:
    aliases: list[UnitTypeRef]
    super_type: UnitTypeRef
    prefix: str
    suffix: str
    separator: str


@define(slots=True)
class UnitTypePool:
    types: list[UnitType]


@define(slots=True)
class UnitValue:
    value: _value_type = field(validator=validators.instance_of(_value_type))
    
    # @value.validator
    # def _check_value(self, attribute: Any, value: _value_type) -> None:
    #     if not isinstance(value, _value_type):
    #         raise TypeError(f"Expected {_value_type}, got {type(value)}")


@define(slots=True)
# @validate
class Unit:
    type_ref: UnitTypeRef or str = field(validator=validators.instance_of(UnitTypeRef | str))
    
    @type_ref.validator
    def _check_type_ref(self, attribute: Any, value: UnitTypeRef | str) -> None:
        if isinstance(value, str):
            return
        if isinstance(value, UnitTypeRef):
            return
        if not isinstance(value, UnitTypeRef):
            raise TypeError(f"Expected {UnitTypeRef}, got {type(value)}")
        
    value: UnitValue or str = field(validator=validators.instance_of(type(UnitValue) | str))
    
    @value.validator
    def _check_value(self, attribute: Any, value: UnitValue | str) -> None:
        if isinstance(value, str):
            return
        if isinstance(value, UnitValue):
            return
        if not isinstance(value, UnitValue):
            raise TypeError(f"Expected {UnitValue}, got {type(value)}")
        

@define(slots=True)
class Generator:
    unit_type_pool: UnitTypePool

    @frozen
    class FrozenUnitValue(UnitValue):
        pass

    def create_unit_value(self, value: _value_type | UnitValue, type_ref: Optional[UnitTypeRef] = None) -> UnitValue or FrozenUnitValue:

        return self.FrozenUnitValue(value)
    
    @frozen
    class FrozenUnit(Unit):
        pass
    
    def create_unit(self, type_ref: UnitTypeRef | str, value: UnitValue | str) -> Unit or FrozenUnit:
        return self.FrozenUnit(type_ref, value)


unit_type_pool = UnitTypePool(types := [UnitType(aliases := [UnitTypeRef("new_type")], super_type := UnitTypeRef("string"), prefix := "new_type:", suffix := ":new_type", separator := "")])
print(unit_type_pool)

create_unit_type_ref = UnitTypeRef(type_ref="str")
print(create_unit_type_ref.__slots__)

create_base_unit = Unit(type_ref := UnitTypeRef("new_type"), value := UnitValue("unit_01"))
print(create_base_unit.__slots__)

generator = Generator(unit_type_pool)
frozen_unit_value = generator.create_unit_value("unit_01")
print(frozen_unit_value)

try:
    frozen_unit_value.value = generator.create_unit_value("new_value_two")
except Exception as e:
    print(e.__class__.__name__, e.args)

frozen_unit = generator.create_unit("new_type", "new_value2")
print(frozen_unit)

frozen_unit_2 = generator.create_unit("new_type", "new_value_2")
print(frozen_unit_2)

# frozen_unit_using_freeze = freeze(Unit(type_ref := UnitTypeRef("new_type"), value := UnitValue("unit_02")))
# print(frozen_unit_using_freeze)

print(frozen_unit_2.__slots__)
print(frozen_unit_2.__dir__())

UnitTypePool(types=[UnitType(aliases=[UnitTypeRef(type_ref='new_type')], super_type=UnitTypeRef(type_ref='string'), prefix='new_type:', suffix=':new_type', separator='')])
('type_ref', '__weakref__')


TypeError: ("'value' must be type | str (got UnitValue(value='unit_01') that is a <class '__main__.UnitValue'>).", Attribute(name='value', default=NOTHING, validator=_AndValidator(_validators=(<instance_of validator for type type | str>, <function Unit._check_value at 0x110989f80>)), repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class '__main__.UnitValue'>, converter=None, kw_only=False, inherited=False, on_setattr=None, alias='value'), type | str, UnitValue(value='unit_01'))